<a href="https://colab.research.google.com/github/gothammered/2022-2-Python/blob/main/HW1_SubwayNavigation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hello world

In [87]:
"""
Using xlrd is a good way to read excel file.
However, because not only the package is no longer maintained but also can only read xls file, which is also deprecated, this code used pandas instead.
(Please refer to https://github.com/python-excel/xlrd for further information regarding xlrd package)
"""

# Instead of
# import xlrd
# this code will
import pandas as pd
# because of the reasons mentioned above.

# in order to break code,
import sys

"""
Back-end(?) Part
"""
# 0. Data load and preprocessing
# 1) read excel file
print('Loading...\n')
df_en = pd.read_excel('./simplified_subway_info_english.xlsx')
df_ko = pd.read_excel('./simplified_subway_info_korean.xlsx')

# 2) Convert each columns to list, as shown in the provided sample codes.
#    For loop is required to do this because the pandas package does not provide such exact function.

# 2-1) Retrieve a list of column names for each DataFrame
df_en_columnList = df_en.columns.tolist()
df_ko_columnList = df_ko.columns.tolist()

# 2-2) Retrieve a list of station names for each columns from respective DataFrame
#      (Optional) We can use tqdm function from tqdm package to see how much iterations have passed in for loop
from tqdm import tqdm

# var list should be declared global
data_en = []
data_ko = []

# NAs must be dropped before converting to list
for line in tqdm(df_en_columnList, desc='Loading information(EN)...'):
    data_en.append(df_en[line].dropna().tolist())

for line in tqdm(df_ko_columnList, desc='Loading information(KO)...'):
    data_ko.append(df_ko[line].dropna().tolist())

# print blank row for better readability
print('\n')
print('Program loading successful!')
print('\n')


"""
Front-end(?) part
"""
# main function
def main():
    print('*********' * 10)
    print('Please select an option...')
    print('1. Display subway line information (Line 1 - 4)')
    print('2. Display subway station information')
    print('3. Exit')
    print('*********' * 10)

    # Get user input(choose menu)
    opt = input('Please choose one of the options (1 - 3): ')
    
    # print blank row for better readability
    print('\n')    

    # Error exception
    # The best is using try except, but I deliberately avoided using it
    # 49, 50, 51 are ASCII Decimal codes for '1', '2', '3', respectively
    # get only the first letter from string to avoid error from ord()
    # if blank, run the query again
    if len(opt) == 0:
        print('Nothing was entered... Please try again...')
        main()
        return
    
    if ord(opt[0]) == 48:
        askSure()
        return
    
    elif ord(opt[0]) == 49:
        print('\nRunning option 1...')
        subwayLineQuery()
        return

    elif ord(opt[0]) == 50:
        print('\nRunning option 2...')
        stationNameQuery()
        return

    elif ord(opt[0]) == 51:
        print('\n')
        askSure(main)
    
    else:
        print('\nThe option "{0}" does not exist... Please try again...'.format(opt))
        main()
        return

# When selected option 1 (Line information)
def subwayLineQuery():
    print('*********' * 10)
    print('Subway line information service')
    print('*********' * 10)
    lineNum = input('Please enter a subway line number (1 - 4), or enter 0 to return: ')

    # print blank row for better readability
    print('\n')

    # Error exception
    # The best is using try except, but I deliberately avoided using it as the same reason as above
    if ord(lineNum[0]) == 48:
        main()
        return

    elif ord(lineNum[0]) in [49, 50, 51, 52]:
        displayLineInfo(int(lineNum))
        return

    else:
        print('\nThe line "{0}" does not exist... Please try again...'.format(lineNum))
        subwayLineQuery()
        return

# When selected option 2 (Station information)
def stationNameQuery():
    print('*********' * 10)
    print('Subway station information service')
    print('*********' * 10)
    stationName = input('Please enter a subway station name, or enter 0 to return: ')

    # print blank row for better readability
    print('\n')

    # Error exception
    # The best is using try except, but I deliberately avoided using it as the same reason as above
    if ord(stationName[0]) == 48:
        main()
        return

    # if blank, run the query again
    elif len(stationName) == 0:
        print('Nothing was entered... Please try again...')
        stationNameQuery()
        return

    # For 
    else:
        displayStationInfo(stationName)
        return

# Now that we have station list, 
# 1. Display subway line information
def displayLineInfo(lineNum):
    # We could just use
    # print(data_en[0])
    # print(data_ko[0])
    # for simple implementation, but I used zip for better readability
    for ko, en in zip(data_ko[lineNum - 1], data_en[lineNum - 1]):
        print(ko, '\t', en, sep='')

    # print blank row for better readability
    print('\n')

    # When done printing, ask user whether to return to main or not
    askReturn(main, displayLineInfo, arg=lineNum)
    return

# 2. Display station information
def displayStationInfo(stationName):
    # First, change all the lower cases from input to upper cases since Python distinguishes the two
    # (User might not always input exact station names)
    stationName = stationName.upper()
    
    # Then, try find the stationName from lists, save line numbers
    # lineList should be declared outside the loop
    lineList = []
    for line in range(4):
        if stationName in data_ko[line]:
            lineList.append(line)
        if stationName in data_en[line]:
            lineList.append(line)

    # if lineList is blank, print that no such station name was found
    if len(lineList) == 0:
        print('No such station name was found... Please try again')
        stationNameQuery()
        return

    # if lineList is not blank, print line numbers
    # if only one line was found, print one line(without 'and')
    elif len(lineList) == 1:
        print('{0} station is in Line {1}'.format(stationName, lineList[0] + 1))

    # if multiple lines were found, print those lines with sep 'and'
    else:
        lines = ''
        for line in lineList:
            lines += '{0} and '.format(line + 1)
        
        # delete the last ' and ' from lines
        lines = lines[:-5]
        print('{0} station is in Line {1}'.format(stationName, lines))

    # print blank row for better readability
    print('\n')
    
    # When done printing, ask user whether to return to main or not
    askReturn(main, displayLineInfo, arg=stationName)
    return






# Used when to ask return
# Should be used for functions with SINGLE argument... further implementation is required for multible arguments
def askReturn(fun_opt1, fun_opt2, arg=None, desc_ask='Return to main menu?', desc_opt1='Yes', desc_opt2='No'):
    # Ask whether the user wants to return or stay
    # Descriptions could be altered for further implementation
    print('*********' * 10)
    print(desc_ask)
    print('1. {0}'.format(desc_opt1))
    print('2. {0}'.format(desc_opt2))
    print('*********' * 10)
    opt = input('Please choose one of the options (1 - 2): ')
    
    # print blank row for better readability
    print('\n')
    
    # Error exception
    if ord(opt[0]) == 49:
        # A function can be called like this in python
        # THANK YOU PYTHON
        fun_opt1()
        return
    
    elif ord(opt[0]) == 50:
        # THANK YOU PYTHON, AGAIN
        # If no argument is declared, run the latter function with no arguments
        if arg == None:
            fun_opt2()
            return
        # Else, put argument to re-play the latter function
        else:
            fun_opt2(arg)
            return

    else:
        # If no such option exists, ask the user again
        print('\nThe option "{0}" does not exist... Please try again...'.format(opt))
        askReturn(fun_opt1, fun_opt2, arg, desc_ask, desc_opt1, desc_opt2)
        return

# Used when to ask sure to exit
def askSure(before, arg=None):
    print('*********' * 10)
    print('Do you really want to exit the program?')
    print('1. Yes')
    print('2. No')
    print('*********' * 10)
    opt_exit = input('Please choose one of the options (1 - 2): ')

    # print blank row for better readability
    print('\n')

    # Error exception
    if ord(opt_exit[0]) == 49:
        print('\nExiting the program... Thank you for using!')
        sys.exit()

    # If the user input 2, return to former function
    elif ord(opt_exit[0]) == 50:
        print('\nReturning...')
        if arg == None:
            before()
        else:
            before(arg)
        return

    else:
        print('\nThe option "{0}" does not exist... Please try again...'.format(opt_exit))
        askSure(before, arg)
        return


# RUN!
main()


Converting DataFrame to lists(ko): 100%|██████████| 4/4 [00:00<00:00, 2817.33it/s]




******************************************************************************************
Please select an option...
1. Display subway line information (Line 1 - 4)
2. Display subway station information
3. Exit
******************************************************************************************
Please choose one of the options (1 - 3): 


Nothing was entered... Please try again...
******************************************************************************************
Please select an option...
1. Display subway line information (Line 1 - 4)
2. Display subway station information
3. Exit
******************************************************************************************
Please choose one of the options (1 - 3): 3




******************************************************************************************
Do you really want to exit the program?
1. Yes
2. No
******************************************************************************************
Please choose one of 

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
